## Preparar entorno y credenciales

### Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### GEE

In [ ]:
# Instalar geemap (si no está instalado)
!pip install geemap --upgrade

import geemap
import ee
ee.Authenticate()
ee.Initialize(project="ee-freddyvillota")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.4 MB/s eta 0:00:00


## Modelado

### **Proyección del balance hídrico 2024–2050**

**Descargar datos de precipitación mensual CMIP6 (2025–2050)**

**Dibujar el bounding box de Carchi en un mapa**

In [ ]:
import geemap
import ee

# Cargar el polígono del nivel 1 de Ecuador (departamentos)
carchi_fc = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Ecuador')) \
    .filter(ee.Filter.eq('ADM1_NAME', 'Carchi'))

# Obtener el bounding box (cuadro que lo encierra)
carchi_bbox = carchi_fc.geometry().bounds()

# Crear el mapa centrado en Carchi
Map = geemap.Map(center=[0.5, -77.8], zoom=8)
Map.addLayer(carchi_fc, {'color': 'blue'}, 'Carchi')
Map.addLayer(carchi_bbox, {'color': 'red'}, 'Bounding Box')
Map

Map(center=[0.5, -77.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

**Obtener las coordenadas del bounding box**

In [ ]:
# Extraer coordenadas del bounding box en formato [N, W, S, E]
coords = carchi_bbox.coordinates().getInfo()[0]
lats = [pt[1] for pt in coords]
lons = [pt[0] for pt in coords]

north = max(lats)
south = min(lats)
east = max(lons)
west = min(lons)

print(f"[{north:.2f}, {west:.2f}, {south:.2f}, {east:.2f}]  # [Norte, Oeste, Sur, Este]")

[1.19, -78.55, 0.36, -77.53]  # [Norte, Oeste, Sur, Este]


**Instalar cdsapi para conectar y descargar datos climáticos futuros**

In [ ]:
!pip install cdsapi

In [ ]:
import shutil

# Copiar .cdsapirc al directorio que espera cdsapi
ruta_drive = '/content/drive/MyDrive/secrets/.cdsapirc'
ruta_destino = '/root/.cdsapirc'
shutil.copyfile(ruta_drive, ruta_destino)

'/root/.cdsapirc'

**Generar código en la página de CDS**

✅ Usar el API actualizado desde la web oficial

1. 🔧 Paso 1: Accede a la interfaz gráfica del CDS para CMIP6
👉 https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip6?tab=form

2. 🛠 Paso 2: Llena el formulario así:
Temporal resolution: Monthly
  - Variable: Precipitation
  - Experiment: SSP2-4.5
  - Model: el que prefieras (ej. CNRM-ESM2-1)
  - Ensemble member: r1i1p1f1
  - Period: 2025-01 to 2050-12
  - Region: para Carchi: [1.5, -79.0, 0.2, -76.0])
  - Marca "NetCDF" como formato

3. 🧾 Paso 3: Haz clic en "Show API request"
Esto te dará el código exacto con la sintaxis actualizada y válida como esta:

**PRECIPITACIÓN**

In [ ]:
# import cdsapi

# dataset = "projections-cmip6"
# request = {
#     "temporal_resolution": "monthly",
#     "experiment": "ssp2_4_5",
#     "variable": "surface_downwelling_shortwave_radiation", # northward_wind, precipitation, near_surface_air_temperature, surface_downwelling_shortwave_radiation, eastward_wind
#     "model": "cnrm_esm2_1",
#     "year": [
#         "2015", "2016", "2017",
#         "2018", "2019", "2020",
#         "2021", "2022", "2023",
#         "2024", "2025", "2026",
#         "2027", "2028", "2029",
#         "2030", "2031", "2032",
#         "2033", "2034", "2035",
#         "2036", "2037", "2038",
#         "2039", "2040", "2041",
#         "2042", "2043", "2044",
#         "2045", "2046", "2047",
#         "2048", "2049"
#     ],
#     "month": [
#         "01", "02", "03",
#         "04", "05", "06",
#         "07", "08", "09",
#         "10", "11", "12"
#     ],
#     'area': [1.5, -79.0, 0.2, -76.0],
# }

# client = cdsapi.Client()
# client.retrieve(dataset, request).download("cmip6_shortwaveradiation_2025_2050.zip")

**Descomprimir el archivo**

Esta guardado temporalmente en drive

In [ ]:
# # Extraer para precipitation
# import zipfile

# zip_path = '/content/690c3e522ad7f8aaed47d7437d91b349.zip'
# extract_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02/data/cmip6_precip_data'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)


# # Extraer para shortwave radiation
# import zipfile

# zip_path = '/content/cmip6_shortwaveradiation_2025_2050.zip'
# extract_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02/data/cmip6_radiation_data'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# # Extraer para northward wind
# import zipfile

# zip_path = '/content/cmip6_northwind_2025_2050.zip'
# extract_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02/data/cmip6_northwind_data'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# # Extraer para near-surface air temperature
# import zipfile

# zip_path = '/content/cmip6_temp_2025_2050.zip'
# extract_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02/data/cmip6_temp_data'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# # Extraer para eastward wind
# import zipfile

# zip_path = '/content/cmip6_wind_2025_2050.zip'
# extract_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02/data/cmip6_eastwardwind_data'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

**Verifica el nombre del archivo .nc**

In [ ]:
import os

# Lista de archivos extraídos
#os.listdir(extract_path)

**Procesar archivo NetCDF a DataFrame**

In [ ]:
import os
import xarray as xr
import pandas as pd
from functools import reduce

# Ruta base
base_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02/data'

# Mapeo carpeta → nombre de variable
var_map = {
    'cmip6_precip_data': 'precip_mm',
    'cmip6_temp_data': 'temp_c',
    'cmip6_eastwardwind_data': 'wind_u',
    'cmip6_northwind_data': 'wind_v',
    'cmip6_radiation_data': 'solar_rad'
}

df_list = []

for folder, col_name in var_map.items():
    folder_path = os.path.join(base_path, folder)
    nc_file = [f for f in os.listdir(folder_path) if f.endswith('.nc')][0]
    full_path = os.path.join(folder_path, nc_file)

    ds = xr.open_dataset(full_path)

    # Excluir variables como time_bounds
    var_candidates = [v for v in ds.data_vars if 'time' in ds[v].dims and 'bounds' not in v.lower()]
    var_name = var_candidates[0]

    da = ds[var_name]

    # Promediar sobre todas las dimensiones menos el tiempo
    dims_to_avg = [d for d in da.dims if d != 'time']
    if dims_to_avg:
        da = da.mean(dim=dims_to_avg)

    # Extraer tiempo y valores
    time = da['time'].values
    values = da.values

    # Verificación de longitudes
    if len(time) != len(values):
        print(f"❗ Tamaños distintos para {col_name}: fechas={len(time)}, valores={len(values)}. Saltando esta variable.")
        continue

    df_var = pd.DataFrame({
        'date': pd.to_datetime(time),
        col_name: values
    })
    df_var['year'] = df_var['date'].dt.year
    df_var['month'] = df_var['date'].dt.month

    df_list.append(df_var)

# Fusionar todos los DataFrames
df_cmip6 = reduce(lambda left, right: pd.merge(left, right, on=['date', 'year', 'month']), df_list)
df_cmip6.head()

,date,precip_mm,year,month,temp_c,wind_u,wind_v,solar_rad
0,2015-01-16 12:00:00,0.000109,2015,1,291.423828,-9.422890,0.641277,195.437805
1,2015-02-15 00:00:00,0.000185,2015,2,291.435791,-7.968413,0.566743,193.973434
2,2015-03-16 12:00:00,0.000312,2015,3,291.304688,-4.723524,0.570539,151.706879
3,2015-04-16 00:00:00,0.000331,2015,4,291.405762,-2.849622,0.169271,150.485321
4,2015-05-16 12:00:00,0.000273,2015,5,290.719208,-1.718866,0.157390,154.813721


**Cambiar unidades a las variables climáticas**

**Detalles técnicos confirmados**

1. Temperatura: K a °C ✔️

2. Precipitación: kg/m²/s = mm/s, y multiplicar por 86400 s/día es lo correcto ✔️

3. Radiación solar:

  - W/m² × segundos del mes (3600 × 24 × n días) → J/m² ÷1e6 → MJ/m² ✔️

Coincide con el tratamiento de ERA5 (J/m² → MJ/m²) ✔️

In [ ]:
# Aplicar conversiones sobre df_cmip6 antes de predicción
df_cmip6['temp_c'] = df_cmip6['temp_c'] - 273.15  # K → °C
df_cmip6['precip_mm'] = df_cmip6['precip_mm'] * 86400  # kg/m²/s → mm/día

# Si el valor es un promedio mensual en W/m², conviértelo a MJ/m²/mes:
# MJ/m² = W/m² * 3600 s/h * 24 h/d * días/mes / 1e6
# Ejemplo:
dias_del_mes = df_cmip6['date'].dt.days_in_month
df_cmip6['solar_rad'] = df_cmip6['solar_rad'] * 3600 * 24 * dias_del_mes / 1e6  # → MJ/m²/mes
df_cmip6.head()

,date,precip_mm,year,month,temp_c,wind_u,wind_v,solar_rad
0,2015-01-16 12:00:00,9.386863,2015,1,18.273834,-9.422890,0.641277,523.460668
1,2015-02-15 00:00:00,15.993876,2015,2,18.285797,-7.968413,0.566743,469.260540
2,2015-03-16 12:00:00,26.977810,2015,3,18.154694,-4.723524,0.570539,406.331694
3,2015-04-16 00:00:00,28.607992,2015,4,18.255768,-2.849622,0.169271,390.057930
4,2015-05-16 12:00:00,23.624590,2015,5,17.569214,-1.718866,0.157390,414.653055


In [12]:
# Definir ruta y nombre del archivo
export_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02/data/df_cmip6.csv'

# Guardar como CSV
df_cmip6.to_csv(export_path, index=False)

print("✅ Archivo exportado correctamente a:")
print(export_path)

✅ Archivo exportado correctamente a:
/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_taller02/data/df_cmip6.csv
